# 4-Manifolds and Log VOA
## Example section companion notebook

Seifert manifolds with four singular fibers can be related to characters of $\mathcal V(p,p')$ VOAs. Theorem 4.7 provides a detailed description of this relation for spherical Seifert manifolds however, the theorem may be applied to pseudospherical Seifert manifolds if special pairs of $\hat{w}$ with non-empty $S_{\hat{w},\vec{\underline{b}}}$ appear. In this example notebook, we replicate the computations done in writing the second paragraph of section 6.2 of arXiv:2201.04640, while at the same time demonstrating the capabilities and main functions of pySeifert: a Sage toolkit for the computation of $\hat{Z}$ invariants for Seifert manifolds and related q-series invariants.

We start by loading pySeifert and defining a Seifert manifold

In [1]:
load("../../pySeifert.sage")
S = Seifert([-1, -1, 2, 1, 3, 1, 3, 2, 3]);S

Seifert manifold with 4 exceptional fibers.
Seifert data:
[-1, -1, 2, 1, 3, 1, 3, 2, 3]
Plumbing Matrix:
[-1  1  1  1  1  0]
[ 1  2  0  0  0  0]
[ 1  0 -3  0  0  0]
[ 1  0  0 -3  0  0]
[ 1  0  0  0 -2  1]
[ 0  0  0  0  1 -2]
D: 1, m: 6, det(M): -9

The plumbing matrix of $S$ does not have unit determinant, but the lattice dilation factor $D$ is one. $S$ is therefore a pseudospherical Seifert manifold. To proceed we compute the set of boundary conditions $\mathcal B$ set and the $\vec s$ values for $A_1$.

In [4]:
B = S.boundary_conditions(["A",1], basis="root");B

[[(-1), (1/2), (1/2), (1/2), (0), (1/2)],
 [(0), (1/2), (-1/2), (1/2), (-1), (1/2)],
 [(0), (1/2), (1/2), (-1/2), (-1), (1/2)],
 [(1), (1/2), (-3/2), (-3/2), (-1), (1/2)]]

$S$ admits four independent boundary conditions, for each we can compute the $\vec s$ values with non-empty $S_{\hat{w},\vec{\underline{b}}}$. 

In [5]:
WG = weyl_group(["A",1])
WL = [det(g) for g in WG]
wh_l = list(itertools.product(*[WG if d == 1 else [matrix(1)] for d in S.deg]))
wl_l = list(itertools.product(*[WL if d == 1 else [1] for d in S.deg]))

rho = weyl_vector(["A",1])
wh_b_l = list()
wl_b_l = list()
for b in B:
    wh_b_l.append(list())
    wl_b_l.append(list())
    for wh,wl in zip(wh_l,wl_l):
        whr = [w*rho for w in wh]
        if S.S_set(["A",1],whr,b,basis = "root") != []:
            wh_b_l[-1].append(wh)
            wl_b_l[-1].append(wl)
wh_b_l            

[[([0], [1], [-1], [-1], [0], [-1]), ([0], [-1], [-1], [-1], [0], [-1])],
 [],
 [],
 [([0], [1], [1], [1], [0], [-1]),
  ([0], [1], [-1], [-1], [0], [1]),
  ([0], [-1], [1], [1], [0], [-1]),
  ([0], [-1], [-1], [-1], [0], [1])]]

Only two of the four boundary conditions have non-empty $S_{\hat{w},\vec{\underline{b}}}$ sets. For these, we can identify pairs $\hat{w},\hat{w}'$ such that $\hat{w}_{v_1}=-\hat{w}'_{v_1}$ and $\hat{w}_{v_i}=\hat{w}'_{v_i},\ i=2,3,4$ where $v_i$ are nodes of degree one. This pairing allows for the same arguments of Theorem 4.7 to be applied to this pseudospherical case, and, therefore, we can write the $\hat{Z}$ invariant in terms of $\mathcal{V}(p,p')$ characters.

Because the pairing $\hat{w}\leftrightarrow \hat{w}'$ exists only by choosing  $\hat{w}_{v_1}=-\hat{w}'_{v_1}$, the only choice of $p$ is $p=p_1=2$, $p'=m/p=3$. We can then compare the triplet characters to the $\hat{Z}$ integrands. 

For the trivial boundary condition we get:

In [10]:
n = 0
p = S.p[n]
pp = S.m/p
r = 1
Chp = 0
Trip = 0
expMax = 100
delta = S.delta(["A",1])
s_list = list()
for wh,l in zip(wh_b_l[0],wl_b_l[0]):
    l_red = [li for li,d in zip(l,S.deg) if d == 1]
    Chp += product(l_red)*S.chi_prime_4f_sph(wh,expMax,basis="root")
    if l_red[n] == 1:
        s = abs(pp*(sum([li*sgn(qi)/pi for pi,qi,li in zip(S.p,S.q,l_red) if pi != p])))
        epss = sign(S.q[n]*sum([li*sgn(qi)/pi for pi,qi,li in zip(S.p,S.q,l_red) if pi != p]))
        s_list.append(abs(s))
        Trip += product(l_red)*epss*triplet_character_p_pprime(p,pp,s,r,expMax-delta)
print(f"p: {p}")
print(f"p': {pp}")
print(f"Ẑ integrand:\n {Chp}")
print(f"Triplet character:\n {Trip}")
print(f"Ẑ integrand - q^delta * triplet character: {expand(Chp-q^delta*Trip)}")

p: 2
p': 3
Ẑ integrand:
 q^(1015/12)*z^8/(z^2 + 1/z^2 - 2) - q^(1015/12)*z^6/(z^2 + 1/z^2 - 2) - q^(1015/12)/((z^2 + 1/z^2 - 2)*z^6) + q^(1015/12)/((z^2 + 1/z^2 - 2)*z^8) - q^(763/12)*z^8/(z^2 + 1/z^2 - 2) + q^(763/12)*z^6/(z^2 + 1/z^2 - 2) + q^(763/12)/((z^2 + 1/z^2 - 2)*z^6) - q^(763/12)/((z^2 + 1/z^2 - 2)*z^8) + q^(547/12)*z^6/(z^2 + 1/z^2 - 2) - q^(547/12)*z^4/(z^2 + 1/z^2 - 2) - q^(547/12)/((z^2 + 1/z^2 - 2)*z^4) + q^(547/12)/((z^2 + 1/z^2 - 2)*z^6) - q^(367/12)*z^6/(z^2 + 1/z^2 - 2) + q^(367/12)*z^4/(z^2 + 1/z^2 - 2) + q^(367/12)/((z^2 + 1/z^2 - 2)*z^4) - q^(367/12)/((z^2 + 1/z^2 - 2)*z^6) + q^(223/12)*z^4/(z^2 + 1/z^2 - 2) - q^(223/12)*z^2/(z^2 + 1/z^2 - 2) - q^(223/12)/((z^2 + 1/z^2 - 2)*z^2) + q^(223/12)/((z^2 + 1/z^2 - 2)*z^4) - q^(115/12)*z^4/(z^2 + 1/z^2 - 2) + q^(115/12)*z^2/(z^2 + 1/z^2 - 2) + q^(115/12)/((z^2 + 1/z^2 - 2)*z^2) - q^(115/12)/((z^2 + 1/z^2 - 2)*z^4) + q^(43/12)*z^2/(z^2 + 1/z^2 - 2) - 2*q^(43/12)/(z^2 + 1/z^2 - 2) - q^(7/12)*z^2/(z^2 + 1/z^2 - 2) + q^(43/12

In [11]:
n = 0
p = S.p[n]
pp = S.m/p
r = 1
Chp = 0
Trip = 0
expMax = 100
delta = S.delta(["A",1])
s_list = list()
for wh,l in zip(wh_b_l[-1],wl_b_l[-1]):
    l_red = [li for li,d in zip(l,S.deg) if d == 1]
    Chp += product(l_red)*S.chi_prime_4f_sph(wh,expMax,basis="root")
    if l_red[n] == 1:
        s = abs(pp*(sum([li*sgn(qi)/pi for pi,qi,li in zip(S.p,S.q,l_red) if pi != p])))
        epss = sign(S.q[n]*sum([li*sgn(qi)/pi for pi,qi,li in zip(S.p,S.q,l_red) if pi != p]))
        s_list.append(abs(s))
        Trip += product(l_red)*epss*triplet_character_p_pprime(p,pp,s,r,expMax-delta)
print(f"p: {p}")
print(f"p': {pp}")
print(f"Ẑ integrand:\n {Chp}")
print(f"Triplet character:\n {Trip}")
print(f"Ẑ integrand - q^delta * triplet character: {expand(Chp-q^delta*Trip)}")

p: 2
p': 3
Ẑ integrand:
 -2*q^(369/4)*z^8/(z^2 + 1/z^2 - 2) + 4*q^(369/4)/(z^2 + 1/z^2 - 2) + 2*q^(309/4)*z^8/(z^2 + 1/z^2 - 2) - 2*q^(369/4)/((z^2 + 1/z^2 - 2)*z^8) - 4*q^(309/4)/(z^2 + 1/z^2 - 2) + 2*q^(281/4)*z^6/(z^2 + 1/z^2 - 2) - 4*q^(281/4)/(z^2 + 1/z^2 - 2) + 2*q^(309/4)/((z^2 + 1/z^2 - 2)*z^8) + 2*q^(281/4)/((z^2 + 1/z^2 - 2)*z^6) - 2*q^(229/4)*z^6/(z^2 + 1/z^2 - 2) + 4*q^(229/4)/(z^2 + 1/z^2 - 2) - 2*q^(205/4)*z^6/(z^2 + 1/z^2 - 2) + 4*q^(205/4)/(z^2 + 1/z^2 - 2) - 2*q^(229/4)/((z^2 + 1/z^2 - 2)*z^6) + 2*q^(161/4)*z^6/(z^2 + 1/z^2 - 2) - 2*q^(205/4)/((z^2 + 1/z^2 - 2)*z^6) - 4*q^(161/4)/(z^2 + 1/z^2 - 2) + 2*q^(141/4)*z^4/(z^2 + 1/z^2 - 2) - 4*q^(141/4)/(z^2 + 1/z^2 - 2) + 2*q^(161/4)/((z^2 + 1/z^2 - 2)*z^6) + 2*q^(141/4)/((z^2 + 1/z^2 - 2)*z^4) - 2*q^(105/4)*z^4/(z^2 + 1/z^2 - 2) + 4*q^(105/4)/(z^2 + 1/z^2 - 2) - 2*q^(89/4)*z^4/(z^2 + 1/z^2 - 2) + 4*q^(89/4)/(z^2 + 1/z^2 - 2) - 2*q^(105/4)/((z^2 + 1/z^2 - 2)*z^4) + 2*q^(61/4)*z^4/(z^2 + 1/z^2 - 2) - 2*q^(89/4)/((z^2 + 1/z^2 